# Øving 6 - TMA4320

* **Veiledning:** Torsdag 12. mars og torsdag 19. mars, 2020, kl 0815-1000 i H3.
* **Innleveringsfrist:** Fredag 20. mars, 2020, kl 08:00.* 
* **Innleveringsmetode** Følgende to krav er nødvendig for godkjenning
    1. Opplasting av Jupyter Notebook (individuelt) i Blackboard
    2. Svare på Blackboardskjema for de tre kontrollspørsmålene i øvingen

**Innledning.** I 1989 publiserte Lawrence Shampine sammen med sin PhD-student Przemyslaw Bogacki en artikkel om et innbygd Runge-Kutta par av orden 3(2) med gode egenskaper. I denne oppgaven skal du implementere denne metoden og teste den ut på et interessant problem. Dette kan være en god forberedelse til Prosjekt 3.
Et skritt av metoden kan for problemet $\dot{y}=f(t,y)$ formuleres som

\begin{align*}
k_1 &= f(t_n,y_n) \\
k_2 &= f(t_n+\tfrac12 h_n, y_n+\tfrac12 h_nk_1) \\
k_3 &= f(t_n+\tfrac34 h_n, y_n + \tfrac34 h_nk_2) \\[1mm]
y_{n+1} &= y_n + \tfrac19 h_n (2 k_1 + 3k_2 + 4k_3)\\[1mm]
k_4 &= f(t_n + h_n,y_{n+1}) \\[1mm]
z_{n+1} &= y_n + \tfrac{1}{24} h_n (7k_1 + 6k_2 + 8 k_3 + 3 k_4)
\end{align*}

**Implementasjon.** Man kan implementere denne metoden ved å kode opp en funksjon som tar følgende input
* t0: Startpunktet (venstre endepunkt)
* tend: Sluttpunktet (høyre endepunkt)
* y0: Den gitte initialverdien i t0
* f: En funksjon som beskrevet nedenfor
* h0: En omtrentlig første tidsskrittlengde
* tol: En brukerspesifisert toleranse for lokalfeil

Funksjonen returnerer
* T: et numpy array med alle [t0,t1,...,tend], disse velges av funksjonen
* Y: En 2d numpy-matrise der rad n angir løsning ved tid $t=t_n$
* stats: en dictionary over interessant statistikk. For eksempel kan man ha nøklene 'steps' for antall skritt, 'fevals' for antall kall til funksjonen f, og 'rejects' for antall ganger skritt ble forkastet. Man kan selv velge hva slags informasjon man ellers vil legge inn i stats.

Inputfunksjonen f tar to inputargumenter, $t$ og $y$ der $t$ er et reelt tall og $y$ er et numpy-array med
lengde lik problemets dimensjon. Den returnerer et numpy-array av samme dimensjon som y som inneholder $f(t,y)$.

Når man forsøker å ta ett skritt med metoden, så beregner man først $y_{n+1}$ og $z_{n+1}$ fra formlene gitt ovenfor, og danner så et skalart feilestimat

$$
\mathrm{est}_{n+1} = \| y_{n+1} - z_{n+1} \|
$$

I tillegg fins en justeringsformel for skrittlengden fra ett skritt til et annet. Vi setter

$$
    h_{ny} = \alpha\cdot h_n \cdot \left(\frac{tol}{est}\right)^{1/3},\quad \alpha\approx 0.8-0.9
$$

$\alpha$ er en såkalt pessimistfaktor. Hvis den velges for liten så blir man veldig forsiktig, tar kortere skritt enn nødvendig og bruker dermed uforholdsmessig mange skritt. Hvis den velges for stor så kan det bli mange forkastede skritt. Man tar færre skritt, men bruker regnetid på å gjøre om igjen mange skritt.

Algoritmen kan beskrives som følger (tar ikke med statistikkinnsamling)

* Gitt $t_0, t_{end}, y_0, h_0, tol, f$ som input
* $h=h_0$, $n=0$
* $k_1=f(t_0,y_0)$
* **while** $t_{end}-t_n>0$
    * $h=\min(h,t_{end}-t_n)$
    * Beregn $k_2, k_3$ og deretter $y_{n+1}$ fra oppgitt formel
    * $t_{n+1} = t_n+h$
    * Beregn $k_4$ og deretter $z_{n+1}$ fra oppgitt formel
    * $est=\| y_{n+1} - z_{n+1} \|$
    * **if** $est < tol$ (aksepter skritt)
        * $n=n+1$
        * $k_1=k_4$
    * $h_{ny}$ beregnes fra oppgitt formel og sett $h=h_{ny}$.

Merk at enten skrittet forkastes eller aksepteres så brukes samme formel for å oppdatere skrittlengden $h$.

<!---

*Dynamisk allokering.* Et programteknisk problem som må løses er det faktum at du ikke vet størrelsen på arrayene $T$ og $Y$ apriori. Én måte å løse dette på er å bruke dictionaries som mellomlagring. En kan for eksempel starte
med å sette
* Tdict={0 : t0}
* Ydict={0 : y0}
og deretter lagre i Tdict[k] og Ydict[k] når k økes. Til slutt kan man overføre Tdict og Ydict i numpy arrays T og Y.

En annen måte som kanskje er mer effektiv og minne-økonomisk er å bruke numpy-funksjonen append. I denne oppgaven er vi ikke så opptatt av kodens effektivitet og det blir litt opp til hver og en hvordan man vil løse problemet.

-->




**Oppgave 1** Skriv koden og test den ut på det enkle problemet

$$
u'=v,\ v'=-4u,\quad u(0)=1,\ v(0)=0,\ [t_0,t_{end}]=[0,2\pi]
$$

som har eksakt løsning $u(t)=\cos 2t$ og $v(t)=-2\sin 2t$. Merk at her er $y(t)=[u(t),v(t)]^T$.

Lag 3 plott
* Et som viser de to komponentene $u$ og $v$ beregnet av metoden som funksjon av $t$ i samme plott
* Et som viser den virkelige feilen som funksjon av $t$ (bruk kjent eksakt løsning)
* Et som viser hvordan skrittlengden varierer med som funksjon av $t$

Skriv ut statistikk også, dvs antall skritt, funksjonsevalueringer og forkastede skritt.


**Oppgave 2** Anvend funksjonen på Brusselator-problemet nedenfor. Sett pessimistfaktor $\alpha=0.9$,
$t_0=0$, $t_{end}=20$, og $tol=10^{-5}$. La $h0=0.1$.

\begin{align*}
u' &= 1 + u^2 v - 4 u, & u(0)&=1 \\
v' &= 3u - u^2 v,  & v(0)&=2.9
\end{align*}

Lag 3 plott
* Et som plotter $v$ mot $u$ (faseplott) dvs $u$ langs $x$-aksen og $v$ langs $y$-aksen.
* Et som viser de to komponentene $u$ og $v$  som funksjon av $t$ is samme plott
* Et som viser hvordan skrittlengden varierer med som funksjon av $t$


**Kontrollspørsmål:** Du blir her bedt om å rapportere statistikk fra eksperimentet i Oppgave 2.
Antall skritt koden din bruker, antall funksjonsevalueringer (feval) og antall forkastede skritt.

In [16]:
import numpy as np
import matplotlib.pyplot as plt

def BogackiShampine(t0,tend,y0,f,h0,tol, alpha=0.9):
    T = [t0] #steps
    Y = np.empty((len(y0),2)) 
    stats = {"steps": 0, "feval": 1, "rejects": 0}
    h = h0
    n = 0
    Y[0] = y0
    k1 = f(t0, y0)
    while tend-tn1 > 0:
        hn = min(h, tend-tn)
        k2 = f(T[n]+0.5*hn, Y[n]+0.5*hn*k1)
        k3 = f(T[n]+(3/4)*hn, Y[n]+(3/4)*hn*k2)
        Y[n+1] = Y[n] + (1/9)*hn*(2*k1+3*k2+4*k3)
        T[n+1] = T[n] + hn
        Ta.append(tn1)
        k4 = f(Y[n]+hn,Y[n+1])
        zn1 = Y[n] + (1/24)*hn*(7*k1+6*k2+8*k3+3*k4)
        est = np.linalg.norm(Y[n+1]-zn1)
        if est < tol:
            n += 1
            k1 = k4
            stats["steps"] += 1
        hny = alpha*hn*(tol/est)**(1/3)
        stats["feval"] += 3
        stats["rejects"] += 1
    return T,Y,stats

**Kode for Oppgave 1** skrives under her

In [17]:
def LinearTestProblem(t,y):
    return np.array([y[1],-4*y[0]])

def u(t):
    return np.cos(2*t)

def v(t):
    return -2*sin(2*t)

#Setter følgende verdier
t0 = 0
tend = 2*np.pi
y0 = [1,0]
h0 = 0.5 #random
tol = 0.1 #inspirert av neste oppgave
T, Y, stats = BogackiShampine(t0, tend, y0, LinearTestProblem, h0, tol)

print("Stats:\n")
print("Steps taken:",stats["steps"])
print("Function evaluations:", stats["feval"])
print("Rejected steps:", stats["rejects"])

plt.plot(T, Y[0], label="u")
plt.plot(T, Y[1], label="v")
plt.title("Components as a function of t")
plot.legend()
plt.show()

plt.plot(T,np.linalg.norm(Y[0]-u(T)))
plt.title("Real error as a function of t")

#mangler skrittlengde som funksjon av t

/home/ajo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in double_scalars


KeyboardInterrupt: 

**Kode for Oppgave 2** skrives under her

In [6]:
def Brusselator(t,y):
    return np.array([1+y[0]**2*y[1]-4*y[0],3*y[0]-y[0]**2*y[1]])

# Fyll inn koden som løser diffligningen, plotter, og skriver ut statistikk

